In [1]:
import urllib.request 
import gzip
import os
import numpy

In [2]:
def load_dataset():
    def download(filename, source = "http://yann.lecun.com/exdb/mnist/"):
        print ("Downloading: ", filename)
        urllib.request.urlretrieve(source+filename, filename)
        
    def load_mnist_images(filename):
        if not os.path.exists(filename):
            download(filename)
            
        with gzip.open(filename, "rb") as f:
            data = numpy.frombuffer(f.read(), numpy.uint8, offset=16)
            data = data.reshape(-1, 1, 28, 28)
        return data/numpy.float32(256)
        
    def load_mnist_labels(filename):
        if not os.path.exists(filename):
            download(filename)
        with gzip.open(filename, "rb") as f: 
            data = numpy.frombuffer(f.read(), numpy.uint8, offset=8)
        return data
    
    x_train = load_mnist_images("train-images-idx3-ubyte.gz")
    y_train = load_mnist_labels("train-labels-idx1-ubyte.gz")
    x_test = load_mnist_images("t10k-images-idx3-ubyte.gz")
    y_test = load_mnist_labels("t10k-labels-idx1-ubyte.gz")
    
    return x_train, y_train, x_test, y_test

In [3]:
x_train, y_train, x_test, y_test = load_dataset()

In [ ]:
import matplotlib 
from matplotlib import pyplot
%matplotlib notebook

matplotlib.use("TkAgg")
fig = pyplot.imshow(x_train[1][0])
pyplot.show(fig)

In [4]:
import lasagne
import theano
import theano.tensor as T

In [5]:
def build_NN(input_var=None):
    input_layer = lasagne.layers.InputLayer(shape=(None, 1, 28, 28), input_var=input_var)
    input_layer_drop = lasagne.layers.DropoutLayer(input_layer, p=0.2)
    
    hidden_layer_1 = lasagne.layers.DenseLayer(input_layer_drop, num_units=800,
                                    nonlinearity=lasagne.nonlinearities.rectify,
                                       W = lasagne.init.GlorotUniform())
    
    hidden_layer_1_drop = lasagne.layers.DropoutLayer(hidden_layer_1, p=0.5) 
    
    hidden_layer_2 = lasagne.layers.DenseLayer(hidden_layer_1_drop, num_units=800,
                                    nonlinearity=lasagne.nonlinearities.rectify,
                                       W = lasagne.init.GlorotUniform())
    
    hidden_layer_2_drop = lasagne.layers.DropoutLayer(hidden_layer_2, p=0.5)
    
    output_layer = lasagne.layers.DenseLayer(hidden_layer_2_drop, num_units=1, 
                                            nonlinearity = lasagne.nonlinearities.softmax)
    
    return output_layer

In [6]:
input_var = T.tensor4("inputs") #An empty 4-dimensional array
target_var = T.ivector("targets") #An empty 1-dimensional integer array to represent the labels

network = build_NN(input_var) #Initialize the neural network
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
#Categorical cross entropy in one of the standard error functions with classification problems
loss = loss.mean()

#update the weights based on the value of the error function

params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01, momentum = 0.9)

train_function = theano.function([input_var, target_var], loss, updates=updates)

#Calling this function for a certain amount of times will train the neural network

C:\Users\Gamer Fx\Anaconda3\lib\site-packages\theano\tensor\nnet\nnet.py:814: UserWarning: The softmax is applied on a dimension of shape 1, which does not have a semantic meaning.
  warnings.warn("The softmax is applied on a dimension of shape 1, which does not have a semantic meaning.")


ValueError: ('The following error happened while compiling the node', Dot22(Reshape{2}.0, W), '\n', 'invalid token "Fx\\Anaconda3\\Library\\bin"" in ldflags_str: "-L"C:\\Users\\Gamer Fx\\Anaconda3\\Library\\bin" -lmkl_core -lmkl_intel_thread -lmkl_rt"')

In [ ]:
num_training_steps = 10
for step in range(num_training_steps):
    train_err = train_function(x_train, y_train)
    print("Current step is:" + str(step))

In [ ]:
test_prediction = lasagne.layers.get_output(network)
function_val = theano.function([input_var], test_prediction)

func